# PIPEX

## Makefile

In [39]:
%%file ../pipex_working/Makefile
# **************************************************************************** #
#                                                                              #
#                                                         :::      ::::::::    #
#    Makefile                                           :+:      :+:    :+:    #
#                                                     +:+ +:+         +:+      #
#    By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+         #
#                                                 +#+#+#+#+#+   +#+            #
#    Created: 2023/01/30 14:27:11 by dpentlan          #+#    #+#              #
#    Updated: 2023/04/29 12:19:40 by dpentlan         ###   ########.fr        #
#                                                                              #
# **************************************************************************** #

NAME = pipex
BONUS= pipex_bonus
RED='\033[0;31m'
GREEN='\033[0;32m'
NOCOLOR='\033[0m'

# ***** Sources *****

SRCS = pipex.c \
	pipex_utils_1.c \
	pipex_utils_2.c \
	pipex_path_utils_1.c \
	pipex_fork_utils_1.c \

SRCS_DIR = mandatory/srcs/

SRCS_BONUS = pipex_bonus.c \
	pipex_bonus_utils_1.c \
	pipex_bonus_utils_2.c \
	pipex_bonus_path_utils_1.c \
	pipex_bonus_fork_utils_1.c \
	pipex_bonus_heredoc_utils_1.c \

SRCS_DIR_BONUS = bonus/srcs/

# ***** Includes *****

INCS = pipex.h

INCS_DIR = mandatory/includes/

INCS_BONUS = pipex.h

INCS_DIR_BONUS = bonus/includes/

# ***** Objects *****

OBJS = $(patsubst %.c,$(OBJS_DIR)%.o,$(SRCS))

OBJS_DIR = mandatory/objs/

OBJS_BONUS = $(patsubst %.c,$(OBJS_DIR_BONUS)%.o,$(SRCS_BONUS))

OBJS_DIR_BONUS = bonus/objs/

# ***** Compile and Archiver *****

CFLAGS = -Wall -Wextra -Werror -g

CC = cc

LIBFT = -Llibft -lft

# ***** Build *****

all: $(NAME)

bonus: $(BONUS)

$(NAME): $(OBJS)
	@echo -e -n ${GREEN}
	@make -C libft --silent
	@echo -e ===== MAKING PIPEX =====
	@echo -e -n ${NOCOLOR}
	@$(CC) $(CFLAGS) $(OBJS) $(LIBFT) -o $(NAME)

$(BONUS): $(OBJS_BONUS)
	@echo -e -n ${GREEN}
	@make -C libft --silent
	@echo -e ===== MAKING PIPEX =====
	@echo -e -n ${NOCOLOR}
	@$(CC) $(CFLAGS) $(OBJS_BONUS) $(LIBFT) -o $(BONUS)

$(OBJS_DIR)%.o: $(SRCS_DIR)%.c
	@mkdir -p $(OBJS_DIR)
	@$(CC) -c $(CFLAGS) -I$(INCS_DIR) $< -o $@

$(OBJS_DIR_BONUS)%.o: $(SRCS_DIR_BONUS)%.c
	@mkdir -p $(OBJS_DIR_BONUS)
	@$(CC) -c $(CFLAGS) -I$(INCS_DIR_BONUS) $< -o $@

clean:
	@make -C libft clean --silent
	@rm -f $(OBJS)
	@rm -rf $(OBJS_DIR)
	@rm -f $(OBJS_BONUS)
	@rm -rf $(OBJS_DIR_BONUS)

fclean: clean
	@make -C libft fclean --silent
	@rm -f $(NAME)
	@rm -f $(BONUS)
	@rm -f gdb.txt

re: fclean all


Overwriting ../pipex_working/Makefile


# ~M~A~N~D~A~T~O~R~Y~

## HEADERS

In [ ]:
%%file ../pipex_working/mandatory/includes/pipex.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex.h                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:57:04 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/20 12:57:37 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef PIPEX_H
# define PIPEX_H

# include "../../libft/includes/libft.h"
# include <unistd.h>
# include <sys/wait.h>
# include <error.h>

typedef struct s_pipex {
	int			pipe[2];
	pid_t		pid;
	int			argc;
	int			prev_pipe;
	int			com_num;
	int			temp_used;
	char		**argv;
	char		**envp;
	char		*cmd_abspath;
	char		**cmd_args;
	char		**path_tab;
}				t_pipex;

//	pipex.c
//	int main (int argc, char **argv, char **envp);

//	pipex_utils_1.c
int		px_error(t_pipex *pipex, char *err_message);
int		px_infile(t_pipex *pipex);
int		px_outfile(t_pipex *pipex);
void	ft_free_tab(char **table);

//	pipex_utils_2.c
int		px_init(t_pipex *pipex, int argc, char **argv, char **envp);
int		px_exec_args(t_pipex *pipex);
void	px_close_fds(t_pipex *pipex);

//	pipex_path_utils_1.c
void	px_path_tab_add_fw_slash(t_pipex *pipex);
void	px_get_path_tab(t_pipex *pipex);
void	px_build_abspath(t_pipex *pipex);
void	px_get_abspath(t_pipex *pipex);
void	px_get_execargs(t_pipex *pipex);

//	pipex_fork_utils_1.c
void	px_pre_exec_cleanup(t_pipex *pipex);
void	px_child_process(t_pipex *pipex);
void	px_fork_loop(t_pipex *pipex);

//	pipex_heredoc_utils_1.c

#endif


Overwriting ../pipex_working/mandatory/includes/pipex.h


## SOURCES

### pipex.c

In [12]:
%%file ../pipex_working/mandatory/srcs/pipex.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex.c                                            :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:56:25 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/29 16:03:05 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

int	main(int argc, char **argv, char **envp)
{
	t_pipex	*pipex;
	t_pipex	s_pipex;

	pipex = &s_pipex;
	px_init(pipex, argc, argv, envp);
	px_fork_loop(pipex);
	if (pipex->temp_used == -1)
		unlink("/tmp/pipex");
	if (pipex->path_tab)
	{
		ft_free_tab(pipex->path_tab);
		pipex->path_tab = 0;
	}
	px_close_fds(pipex);
	return (0);
}


Overwriting ../pipex_working/mandatory/srcs/pipex.c


### pipex_utils_1.c

In [22]:
%%file ../pipex_working/mandatory/srcs/pipex_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_utils_1.c                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/29 17:20:54 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/09 15:31:54 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

int	px_error(t_pipex *pipex, char *err_message)
{
	if (ft_strnstr(err_message, "args", 4))
		ft_putstr_fd("Usage: pipex [infile] ['command1']\
['command2'] [outfile]\n", 1);
	else if (ft_strnstr(err_message, "no_path", 7))
		ft_putstr_fd("path: No valid path found in envp\n", 1);
	else
		perror(err_message);
	if (pipex->cmd_abspath)
	{
		free(pipex->cmd_abspath);
		pipex->cmd_abspath = 0;
	}
	if (pipex->cmd_args)
	{
		ft_free_tab(pipex->cmd_args);
		pipex->cmd_args = 0;
	}
	if (pipex->path_tab)
	{
		ft_free_tab(pipex->path_tab);
		pipex->path_tab = 0;
	}
	px_close_fds(pipex);
	exit(EXIT_FAILURE);
}

int	px_infile(t_pipex *pipex)
{
	int		in_file;

	in_file = 0;
	if (pipex->temp_used < 0)
	{
		perror(pipex->argv[1]);
		in_file = open("/tmp/pipex", O_CREAT | O_RDONLY, 0666);
		if (in_file < 0)
			px_error(pipex, "open");
		dup2(in_file, STDIN_FILENO);
		close(in_file);
		return (1);
	}
	in_file = open(pipex->argv[1], O_RDONLY);
	if (in_file < 0)
		px_error(pipex, "open");
	dup2(in_file, STDIN_FILENO);
	close(in_file);
	return (0);
}

int	px_outfile(t_pipex *pipex)
{
	int		out_file;
	char	*fn;

	out_file = 0;
	fn = pipex->argv[pipex->argc - 1];
	out_file = open(fn, O_CREAT | O_WRONLY | O_TRUNC, 0666);
	if (out_file < 0)
		px_error(pipex, "open");
	dup2(out_file, STDOUT_FILENO);
	close(out_file);
	return (0);
}

void	ft_free_tab(char **table)
{
	int	i;

	i = 0;
	while (table[i])
	{
		free(table[i]);
		i++;
	}
	free(table);
	return ;
}


Overwriting ../pipex_working/mandatory/srcs/pipex_utils_1.c


### pipex_utils_2.c

In [14]:
%%file ../pipex_working/mandatory/srcs/pipex_utils_2.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_utils_2.c                                    :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/05/12 18:57:46 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/12 18:57:59 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

int	px_init(t_pipex *pipex, int argc, char **argv, char **envp)
{
	pipex->pipe[0] = 0;
	pipex->pipe[1] = 0;
	pipex->prev_pipe = -1;
	pipex->com_num = 0;
	pipex->cmd_abspath = 0;
	pipex->cmd_args = 0;
	pipex->path_tab = 0;
	if (argc < 5 || argc > 5)
		px_error(pipex, "args");
	pipex->argc = argc;
	pipex->argv = argv;
	pipex->envp = envp;
	pipex->temp_used = access(pipex->argv[1], F_OK | R_OK);
	return (0);
}

int	px_exec_args(t_pipex *pipex)
{
	px_get_execargs(pipex);
	px_get_abspath(pipex);
	return (0);
}

void	px_close_fds(t_pipex *pipex)
{
	close(STDIN_FILENO);
	close(STDOUT_FILENO);
	close(STDERR_FILENO);
	close(pipex->pipe[0]);
	close(pipex->pipe[1]);
	if (pipex->prev_pipe != -1)
		close(pipex->prev_pipe);
}


Overwriting ../pipex_working/mandatory/srcs/pipex_utils_2.c


### pipex_path_utils_1.c

In [15]:
%%file ../pipex_working/mandatory/srcs/pipex_path_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_path_utils_1.c                               :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/05/12 18:58:17 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/12 18:58:25 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

void	px_path_tab_add_fw_slash(t_pipex *pipex)
{
	int		i;
	char	*buf;

	i = 0;
	buf = 0;
	while (pipex->path_tab[i])
	{
		if (pipex->path_tab[i][ft_strlen(pipex->path_tab[i]) - 1] != '/')
		{
			buf = ft_strjoin(pipex->path_tab[i], "/");
			free(pipex->path_tab[i]);
			pipex->path_tab[i] = buf;
			if (!pipex->path_tab[i])
				px_error(pipex, "malloc");
		}
		i++;
	}
}

void	px_get_path_tab(t_pipex *pipex)
{
	char	*path;
	char	**env_elem;

	path = 0;
	env_elem = pipex->envp;
	while (*env_elem)
	{
		path = ft_strnstr(*env_elem, "PATH=", 5);
		if (path)
			break ;
		env_elem++;
	}
	if (!*env_elem)
		px_error(pipex, "no_path");
	pipex->path_tab = ft_split(path + 5, ':');
	if (!pipex->path_tab)
		px_error(pipex, "malloc");
	px_path_tab_add_fw_slash(pipex);
}

void	px_build_abspath(t_pipex *pipex)
{
	int	i;

	i = 0;
	while (pipex->path_tab[i])
	{
		if (pipex->cmd_abspath)
			free(pipex->cmd_abspath);
		pipex->cmd_abspath = ft_strjoin(pipex->path_tab[i], pipex->cmd_args[0]);
		if (!pipex->cmd_abspath)
			px_error(pipex, "malloc");
		if (!access(pipex->cmd_abspath, F_OK | X_OK))
			break ;
		i++;
	}
}

void	px_get_abspath(t_pipex *pipex)
{
	if (pipex->cmd_args[0][0] == '/')
	{
		pipex->cmd_abspath = ft_strdup(pipex->cmd_args[0]);
		if (!pipex->cmd_abspath)
			px_error(pipex, "malloc");
	}
	else
	{
		if (!pipex->path_tab)
			px_get_path_tab(pipex);
		px_build_abspath(pipex);
	}
}

void	px_get_execargs(t_pipex *pipex)
{
	pipex->cmd_args = ft_split(pipex->argv[pipex->com_num], ' ');
	if (!pipex->cmd_args)
		px_error(pipex, "malloc");
}


Overwriting ../pipex_working/mandatory/srcs/pipex_path_utils_1.c


### pipex_fork_utils_1.c

In [16]:
%%file ../pipex_working/mandatory/srcs/pipex_fork_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_fork_utils_1.c                               :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/05/12 18:58:31 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/12 18:58:40 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex.h"

void	px_pre_exec_cleanup(t_pipex *pipex)
{
	close(pipex->pipe[0]);
	close(pipex->pipe[1]);
	if (pipex->prev_pipe != -1)
		close(pipex->prev_pipe);
	if (!pipex->cmd_abspath)
	{
		px_close_fds(pipex);
		exit(EXIT_FAILURE);
	}
}

void	px_child_process(t_pipex *pipex)
{
	px_exec_args(pipex);
	if (access(pipex->cmd_abspath, F_OK | X_OK))
		px_error(pipex, pipex->cmd_args[0]);
	if (pipex->com_num == 2)
	{
		px_infile(pipex);
		dup2(pipex->pipe[1], STDOUT_FILENO);
	}
	else if (pipex->com_num != pipex->argc - 2)
	{
		dup2(pipex->prev_pipe, STDIN_FILENO);
		dup2(pipex->pipe[1], STDOUT_FILENO);
	}
	else if (pipex->com_num == pipex->argc - 2)
	{
		px_outfile(pipex);
		dup2(pipex->prev_pipe, STDIN_FILENO);
	}
	px_pre_exec_cleanup(pipex);
	execve(pipex->cmd_abspath, pipex->cmd_args, pipex->envp);
}

void	px_fork_loop(t_pipex *pipex)
{
	pipex->com_num = 2;
	while (pipex->com_num < pipex->argc - 1)
	{
		if (pipe(pipex->pipe))
			px_error(pipex, "pipe");
		pipex->pid = fork();
		if (pipex->pid < 0)
			px_error(pipex, "fork");
		if (pipex->pid == 0)
			px_child_process(pipex);
		else
		{
			close(pipex->pipe[1]);
			if (pipex->prev_pipe != -1)
				close(pipex->prev_pipe);
			pipex->prev_pipe = pipex->pipe[0];
		}
		pipex->com_num++;
		waitpid(pipex->pid, NULL, 0);
	}
}

Overwriting ../pipex_working/mandatory/srcs/pipex_fork_utils_1.c


# ~B~O~N~U~S~

## HEADERS

In [25]:
%%file ../pipex_working/bonus/includes/pipex_bonus.h
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_bonus.h                                      :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:57:04 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/15 15:37:12 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#ifndef PIPEX_BONUS_H
# define PIPEX_BONUS_H

# include "../../libft/includes/libft.h"
# include <unistd.h>
# include <sys/wait.h>
# include <error.h>

typedef struct s_pipex {
	int			pipe[2];
	pid_t		pid;
	int			argc;
	int			prev_pipe;
	int			com_num;
	int			temp_used;
	char		**argv;
	char		**envp;
	char		*cmd_abspath;
	char		**cmd_args;
	char		**path_tab;
}				t_pipex;

//	pipex.c
//	int main (int argc, char **argv, char **envp);

//	pipex_utils_1.c
int		px_error(t_pipex *pipex, char *err_message);
int		px_infile(t_pipex *pipex);
int		px_outfile(t_pipex *pipex);
void	ft_free_tab(char **table);

//	pipex_utils_2.c
int		px_init(t_pipex *pipex, int argc, char **argv, char **envp);
int		px_exec_args(t_pipex *pipex);
void	px_close_fds(t_pipex *pipex);

//	pipex_path_utils_1.c
void	px_path_tab_add_fw_slash(t_pipex *pipex);
void	px_get_path_tab(t_pipex *pipex);
void	px_build_abspath(t_pipex *pipex);
void	px_get_abspath(t_pipex *pipex);
void	px_get_execargs(t_pipex *pipex);

//	pipex_fork_utils_1.c
void	px_pre_exec_cleanup(t_pipex *pipex);
void	px_child_process(t_pipex *pipex);
void	px_fork_loop(t_pipex *pipex);

//	pipex_heredoc_utils_1.c

#endif


Overwriting ../pipex_working/bonus/includes/pipex_bonus.h


## SOURCES

### pipex_bonus.c

In [28]:
%%file ../pipex_working/bonus/srcs/pipex_bonus.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_bonus.c                                      :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/20 12:56:25 by dpentlan          #+#    #+#             */
/*   Updated: 2023/04/29 16:03:05 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex_bonus.h"

int	main(int argc, char **argv, char **envp)
{
	t_pipex	*pipex;
	t_pipex	s_pipex;

	pipex = &s_pipex;
	px_init(pipex, argc, argv, envp);
	px_fork_loop(pipex);
	if (pipex->temp_used == -1)
		unlink("/tmp/pipex");
	if (pipex->path_tab)
	{
		ft_free_tab(pipex->path_tab);
		pipex->path_tab = 0;
	}
	px_close_fds(pipex);
	return (0);
}


Overwriting ../pipex_working/bonus/srcs/pipex_bonus.c


### pipex_bonus_utils_1.c

In [30]:
%%file ../pipex_working/bonus/srcs/pipex_bonus_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_bonus_utils_1.c                              :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/04/29 17:20:54 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/09 15:31:54 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex_bonus.h"

int	px_error(t_pipex *pipex, char *err_message)
{
	if (ft_strnstr(err_message, "args", 4))
		ft_putstr_fd("Usage: pipex [infile] ['command'] ... [outfile]\n", 1);
	else if (ft_strnstr(err_message, "no_path", 7))
		ft_putstr_fd("path: No valid path found in envp\n", 1);
	else
		perror(err_message);
	if (pipex->cmd_abspath)
	{
		free(pipex->cmd_abspath);
		pipex->cmd_abspath = 0;
	}
	if (pipex->cmd_args)
	{
		ft_free_tab(pipex->cmd_args);
		pipex->cmd_args = 0;
	}
	if (pipex->path_tab)
	{
		ft_free_tab(pipex->path_tab);
		pipex->path_tab = 0;
	}
	px_close_fds(pipex);
	exit(EXIT_FAILURE);
}

int	px_infile(t_pipex *pipex)
{
	int		in_file;

	in_file = 0;
	if (pipex->temp_used < 0)
	{
		perror(pipex->argv[1]);
		in_file = open("/tmp/pipex", O_CREAT | O_RDONLY, 0666);
		if (in_file < 0)
			px_error(pipex, "open");
		dup2(in_file, STDIN_FILENO);
		close(in_file);
		return (1);
	}
	in_file = open(pipex->argv[1], O_RDONLY);
	if (in_file < 0)
		px_error(pipex, "open");
	dup2(in_file, STDIN_FILENO);
	close(in_file);
	return (0);
}

int	px_outfile(t_pipex *pipex)
{
	int		out_file;
	char	*fn;

	out_file = 0;
	fn = pipex->argv[pipex->argc - 1];
	out_file = open(fn, O_CREAT | O_WRONLY | O_TRUNC, 0666);
	if (out_file < 0)
		px_error(pipex, "open");
	dup2(out_file, STDOUT_FILENO);
	close(out_file);
	return (0);
}

void	ft_free_tab(char **table)
{
	int	i;

	i = 0;
	while (table[i])
	{
		free(table[i]);
		i++;
	}
	free(table);
	return ;
}


Writing ../pipex_working/bonus/srcs/pipex_bonus_utils_1.c


### pipex_bonus_utils_2.c

In [40]:
%%file ../pipex_working/bonus/srcs/pipex_bonus_utils_2.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_bonus_utils_2.c                              :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/05/12 18:57:46 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/12 18:57:59 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex_bonus.h"

int	px_init(t_pipex *pipex, int argc, char **argv, char **envp)
{
	pipex->pipe[0] = 0;
	pipex->pipe[1] = 0;
	pipex->prev_pipe = -1;
	pipex->com_num = 0;
	pipex->cmd_abspath = 0;
	pipex->cmd_args = 0;
	pipex->path_tab = 0;
	if (argc < 5)
		px_error(pipex, "args");
	pipex->argc = argc;
	pipex->argv = argv;
	pipex->envp = envp;
	pipex->temp_used = access(pipex->argv[1], F_OK | R_OK);
	return (0);
}

int	px_exec_args(t_pipex *pipex)
{
	px_get_execargs(pipex);
	px_get_abspath(pipex);
	return (0);
}

void	px_close_fds(t_pipex *pipex)
{
	close(STDIN_FILENO);
	close(STDOUT_FILENO);
	close(STDERR_FILENO);
	close(pipex->pipe[0]);
	close(pipex->pipe[1]);
	if (pipex->prev_pipe != -1)
		close(pipex->prev_pipe);
}


Overwriting ../pipex_working/bonus/srcs/pipex_bonus_utils_2.c


### pipex_bonus_path_utils_1.c

In [34]:
%%file ../pipex_working/bonus/srcs/pipex_bonus_path_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_bonus_path_utils_1.c                         :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/05/12 18:58:17 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/12 18:58:25 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex_bonus.h"

void	px_path_tab_add_fw_slash(t_pipex *pipex)
{
	int		i;
	char	*buf;

	i = 0;
	buf = 0;
	while (pipex->path_tab[i])
	{
		if (pipex->path_tab[i][ft_strlen(pipex->path_tab[i]) - 1] != '/')
		{
			buf = ft_strjoin(pipex->path_tab[i], "/");
			free(pipex->path_tab[i]);
			pipex->path_tab[i] = buf;
			if (!pipex->path_tab[i])
				px_error(pipex, "malloc");
		}
		i++;
	}
}

void	px_get_path_tab(t_pipex *pipex)
{
	char	*path;
	char	**env_elem;

	path = 0;
	env_elem = pipex->envp;
	while (*env_elem)
	{
		path = ft_strnstr(*env_elem, "PATH=", 5);
		if (path)
			break ;
		env_elem++;
	}
	if (!*env_elem)
		px_error(pipex, "no_path");
	pipex->path_tab = ft_split(path + 5, ':');
	if (!pipex->path_tab)
		px_error(pipex, "malloc");
	px_path_tab_add_fw_slash(pipex);
}

void	px_build_abspath(t_pipex *pipex)
{
	int	i;

	i = 0;
	while (pipex->path_tab[i])
	{
		if (pipex->cmd_abspath)
			free(pipex->cmd_abspath);
		pipex->cmd_abspath = ft_strjoin(pipex->path_tab[i], pipex->cmd_args[0]);
		if (!pipex->cmd_abspath)
			px_error(pipex, "malloc");
		if (!access(pipex->cmd_abspath, F_OK | X_OK))
			break ;
		i++;
	}
}

void	px_get_abspath(t_pipex *pipex)
{
	if (pipex->cmd_args[0][0] == '/')
	{
		pipex->cmd_abspath = ft_strdup(pipex->cmd_args[0]);
		if (!pipex->cmd_abspath)
			px_error(pipex, "malloc");
	}
	else
	{
		if (!pipex->path_tab)
			px_get_path_tab(pipex);
		px_build_abspath(pipex);
	}
}

void	px_get_execargs(t_pipex *pipex)
{
	pipex->cmd_args = ft_split(pipex->argv[pipex->com_num], ' ');
	if (!pipex->cmd_args)
		px_error(pipex, "malloc");
}


Writing ../pipex_working/bonus/srcs/pipex_bonus_path_utils_1.c


### pipex_bonus_fork_utils_1.c

In [36]:
%%file ../pipex_working/bonus/srcs/pipex_bonus_fork_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_bonus_fork_utils_1.c                         :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/05/12 18:58:31 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/12 18:58:40 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex_bonus.h"

void	px_pre_exec_cleanup(t_pipex *pipex)
{
	close(pipex->pipe[0]);
	close(pipex->pipe[1]);
	if (pipex->prev_pipe != -1)
		close(pipex->prev_pipe);
	if (!pipex->cmd_abspath)
	{
		px_close_fds(pipex);
		exit(EXIT_FAILURE);
	}
}

void	px_child_process(t_pipex *pipex)
{
	px_exec_args(pipex);
	if (access(pipex->cmd_abspath, F_OK | X_OK))
		px_error(pipex, pipex->cmd_args[0]);
	if (pipex->com_num == 2)
	{
		px_infile(pipex);
		dup2(pipex->pipe[1], STDOUT_FILENO);
	}
	else if (pipex->com_num != pipex->argc - 2)
	{
		dup2(pipex->prev_pipe, STDIN_FILENO);
		dup2(pipex->pipe[1], STDOUT_FILENO);
	}
	else if (pipex->com_num == pipex->argc - 2)
	{
		px_outfile(pipex);
		dup2(pipex->prev_pipe, STDIN_FILENO);
	}
	px_pre_exec_cleanup(pipex);
	execve(pipex->cmd_abspath, pipex->cmd_args, pipex->envp);
}

void	px_fork_loop(t_pipex *pipex)
{
	pipex->com_num = 2;
	while (pipex->com_num < pipex->argc - 1)
	{
		if (pipe(pipex->pipe))
			px_error(pipex, "pipe");
		pipex->pid = fork();
		if (pipex->pid < 0)
			px_error(pipex, "fork");
		if (pipex->pid == 0)
			px_child_process(pipex);
		else
		{
			close(pipex->pipe[1]);
			if (pipex->prev_pipe != -1)
				close(pipex->prev_pipe);
			pipex->prev_pipe = pipex->pipe[0];
		}
		pipex->com_num++;
		waitpid(pipex->pid, NULL, 0);
	}
}


Writing ../pipex_working/bonus/srcs/pipex_bonus_fork_utils_1.c


### pipex_bonus_heredoc_utils_1.c

In [38]:
%%file ../pipex_working/bonus/srcs/pipex_bonus_heredoc_utils_1.c
/* ************************************************************************** */
/*                                                                            */
/*                                                        :::      ::::::::   */
/*   pipex_bonus_heredoc_utils_1.c                      :+:      :+:    :+:   */
/*                                                    +:+ +:+         +:+     */
/*   By: dpentlan <dpentlan@student.42.fr>          +#+  +:+       +#+        */
/*                                                +#+#+#+#+#+   +#+           */
/*   Created: 2023/05/13 23:46:23 by dpentlan          #+#    #+#             */
/*   Updated: 2023/05/13 23:46:35 by dpentlan         ###   ########.fr       */
/*                                                                            */
/* ************************************************************************** */

#include "pipex_bonus.h"


Writing ../pipex_working/bonus/srcs/pipex_bonus_heredoc_utils_1.c


# TESTING

### Test Build

In [2]:
%%bash
cd ../pipex_working
make re

===== MAKING LIBFT =====
===== MAKING PIPEX =====


In [34]:
%%bash
cd ../pipex_working
make re > /dev/null
./pipex > /dev/null
nm -Duj pipex | grep -v "^_"
make fclean

access@GLIBC_2.2.5
close@GLIBC_2.2.5
dup2@GLIBC_2.2.5
execve@GLIBC_2.2.5
fork@GLIBC_2.2.5
free@GLIBC_2.2.5
malloc@GLIBC_2.2.5
pipe@GLIBC_2.2.5
waitpid@GLIBC_2.2.5
write@GLIBC_2.2.5


In [2]:
%%bash
# use valgrind with --track-fds=all to see if fds are closing appropriately
valgrind --track-fds=all < file1 cat | wc > file2

==86224== Memcheck, a memory error detector
==86224== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.
==86224== Using Valgrind-3.18.1 and LibVEX; rerun with -h for copyright info
==86224== Command: cat
==86224== 
==86224== 
==86224== FILE DESCRIPTORS: 0 open (0 std) at exit.
==86224== 
==86224== HEAP SUMMARY:
==86224==     in use at exit: 0 bytes in 0 blocks
==86224==   total heap usage: 1,051 allocs, 1,051 frees, 205,320 bytes allocated
==86224== 
==86224== All heap blocks were freed -- no leaks are possible
==86224== 
==86224== For lists of detected and suppressed errors, rerun with: -s
==86224== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)


### argument testing

##### args

In [11]:
%%file ./argument_testing_arguments.txt
file1 "cat -e" "/usr/bin/wc -l" file8
file1 cat "/usr/bin/wc -l" file8
file1 "cat -e" wc file8
file1 cat wc file8
file1 "cat -e" "/usr/bin/wc -l" file8
file1 "catlkj -e" "/usr/bin/wc -l" file8
file1 "cat -e" "/usr/bin/wclkj -l" file8
file1 "catlkj -e" "/usr/bin/wclkj -l" file8
file1 cat "/usr/bin/wc -l" file8
file1 catlkj "/usr/bin/wc -l" file8
file1 cat "/usr/bin/wclkj -l" file8
file1 catlkj "/usr/bin/wclkj -l" file8
file1 "cat -e" wc file8
file1 "catlkj -e" wc file8
file1 "cat -e" wclkj file8
file1 "catlkj -e" wclkj file8
file1 cat wc file8
file1 catlkj wc file8
file1 cat wclkj file8
file1 catlkj wclkj file8
file1lksjdlkfj "cat -e" "/usr/bin/wc -l" file8

Overwriting ./argument_testing_arguments.txt


##### script

In [30]:
%%bash
WORKING="../pipex_working/"
TESTING="../pipex_testing/"
ARGS="argument_testing_arguments.txt"

cd $WORKING
rm -f ${WORKING}valgrind.log
rm -f ${TESTING}valgrind.log
make re

cat ${TESTING}$ARGS | while read LINE
do
	rm -f file8
	valgrind --track-fds=all --leak-check=full ./pipex $LINE &>> valgrind.log
	cat file8 &>> valgrind.log
done

echo -n "Total runs		:---------------: " && cat ${TESTING}$ARGS | wc -l
echo -n "Good Valgrind Runs	: Memory	: " && cat valgrind.log | grep "All heap blocks were freed" | wc -l
echo -n "Good Valgrind Runs	: fds		: " && cat valgrind.log | grep "FILE DESCRIPTORS: 0 open (0 std) at exit" | wc -l
echo -n "BAD Valgrind Runs	: fds		: " && cat valgrind.log | grep "Bad file descriptor" | wc -l
echo -n "BAD Valgrind Runs	: Cond J	: " && cat valgrind.log | grep "Conditional jump or move" | wc -l

mv valgrind.log ${TESTING}

Writing val_test.sh


### test vs bash

In [44]:
%%bash
cd ../pipex_working
# working bash
rm -f file8
echo "hi" >> file1
< file1 cat | wc > file8
cat file8
# working pipex
rm -f file8
echo "hi" >> file1
./pipex file1 cat wc file8
cat file8

# broken bash first file
rm -f file8
echo "hi" >> file1
< file1ljk cat | wc > file8
cat file8
# # broken pipex first file
# rm -f file8
# echo "hi" >> file1
# ./pipex file1lkj cat wc file8
# cat file8

# broken bash first cmd
rm -f file8
echo "hi" >> file1
< file1 catlkj | wc > file8
cat file8
# broken pipex first cmd
rm -f file8
echo "hi" >> file1
./pipex file1 catlkj wc file8
cat file8

# broken bash second cmd
rm -f file8
echo "hi" >> file1
< file1 cat | wclkj > file8
cat file8
# broken pipex second cmd
rm -f file8
echo "hi" >> file1
./pipex file1 cat wclkj file8
cat file8

# broken bash second file
rm -f file8
rm -f file8lkj
echo "hi" >> file1
< file1 cat | wc > file8lkj
cat file8lkj
# broken pipex second file
rm -f file8
rm -f file8lkj
echo "hi" >> file1
./pipex file1 cat wc file8lkj
cat file8lkj

    136     136     437
    137     137     440
      0       0       0
      0       0       0
      0       0       0
    143     143     458
    144     144     461


bash: line 16: file1ljk: No such file or directory
bash: line 27: catlkj: command not found
access: No such file or directory
bash: line 38: wclkj: command not found
access: No such file or directory
